In [ ]:
NAME = "LEONARDO PASSOS FIDA"
COLLABORATORS = ""

# Titanic dataset: survival rate analysis

In [ ]:
# IMPORTING LIBRARIES
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
# READING THE CSV FILE
titanic=pd.read_csv('../input/titanic-dataset/train.csv')

In [ ]:
# READING THE HEAD OF THE DATA FRAME TO UNDERSTAND THE COLUMNS AND CATEGORIES
titanic.head()

In [ ]:
# FINDING OUT HOW MUCH DATA WE HAVE IN THE DATA FRAME
titanic.info()

From the .info() method shown above, we can see that the "Age" category has null values, along with the "Cabin" feature.

In [ ]:
# GETTING THE STATISTICAL SUMMARY OF THE DATA
titanic.describe()

Since the non-numeric columns are ommited from the .describe() method, we can get a summary of the non-numeric data using the following method:

In [ ]:
categorical = titanic.dtypes[titanic.dtypes == "object"].index
print(categorical)
titanic[categorical].describe()

First thing to do to clean up the data: finding out if we need all of the variables. Since 'PassengerId' is just a number assigned to each passenger, much like a serial variable in SQL, it is nothing more than an arbitrary identifier, so we can remove it from the data frame.

In [ ]:
titanic.drop(['PassengerId'],axis=1,inplace=True)

Checking the head of the data frame again:

In [ ]:
titanic.head()

The variable 'Survived' definitely must be kept, because it indicates if each passenger lived or died in the tragedy. The main goal is to predict survival. Other variables that are numerical could be useful for predicting survival, such as Pclass, Age, SibSp, Parch, Fare and Embarked, so let's keep all of them for now. There are 3 other variables to consider: Name, Ticket and Cabin. The variable "Name" is a character string of the name of each passenger. Let's analyze the first 15 rows of this column:

In [ ]:
titanic['Name'].sort_values()[0:15]

Checking the properties of the "Name" column:

In [ ]:
titanic['Name'].describe()

The variable Name has 891 unique values,  which is the same as the count of the column. Hence, we know that each name is unique. Married women have their maiden names listed in parentheses. It would be possible to extract last names in an effort to group family members together, but the number of categories would be very large, since there are a lot of people in this column. Additionally, the "Parch" and "SibSp" variables already show us some info about family relationships, so theoretically the variable "Name" could be removed. Nevertheless, since each name is unique, it would be a good idea, from a historical perspective, to keep this variable.

Doing the same procedure with the variable "Ticket":

In [ ]:
titanic['Ticket'][0:15]

In [ ]:
titanic['Ticket'].describe()

The variable "Ticket" has 681 unique values, while the count in this column is 891. The number of unique tickets is not close to the number of passengers, and the ticket numbers do not appear to follow any pattern we could use for grouping people. Hence, this variable can be removed:

In [ ]:
titanic.drop(['Ticket'],axis=1,inplace=True)

Checking the head of the data frame again:

In [ ]:
titanic.head()

Now let's consider the "Cabin" variable. Checking the first 15 rows of this column:

In [ ]:
titanic['Cabin'][0:15]

In [ ]:
titanic['Cabin'].describe()

This column has 147 unique values and the names of the levels for each cabin seem to follow a certain pattern: each starts with a capital letter followed by a number. This structure can be useful to make categories that might be significant for prediction, so this variable can be kept.

The next logical step is to think about transforming any variables. The variable "Survived" is an integer that has only 2 unique values: 0 if a passenger did not survive and 1 if a passenger survived. This is what we call a dummy variable, because 0 indicates absence and 1 represents presence. We could encode this variable as a categorical variable:

In [ ]:
survived = pd.Categorical(titanic["Survived"])
survived = survived.rename_categories(["Died","Survived"])              
survived.describe()

Now, this categorical variable shows appropriate category names. Nevertheless, the variable "Survived" already is well organized in terms of a dummy variable, because we are only interested to extract the information about whether a passenger has survived or not from this variable. Hence, we are not going to change it.

Another variable that can be revisited is "Pclass". It is an integer which has 3 unique values: 1, 2 and 3, which represents first class, second class and third class, respectively. Since the passenger class represents a category, it does not make much sense to keep it as a numeric value. Additionally, class number 1 would be higher than class number 2, which will be higher than class number 3. Nevertheless, when this variable is encoded as only numeric, 1 comes before 2. We can quickly solve this by making a categorical variable:

In [ ]:
Pclass = pd.Categorical(titanic["Pclass"],ordered=True)
Pclass = Pclass.rename_categories(["Class 1","Class 2","Class 3"])              
Pclass.describe()

Now, we can include this variable in the original data frame:

In [ ]:
titanic['Pclass']=Pclass

Checking the head of the data frame to see the chages made so far:

In [ ]:
titanic.head()

We can now focus on the "Cabin" variable. Each cabin appears to be a section of the ship indicated by the capital letter that initiates each value on this column. We can check the unique values:

In [ ]:
titanic["Cabin"].unique()

We can now extract the first letter of each cabin, in order to reduce the number of levels and check the amount of each category of letters for the cabins:

In [ ]:
cabin_letters=titanic['Cabin'].astype(str)
cabin_cat=np.array([cabin[0] for cabin in cabin_letters])
cabin_cat=pd.Categorical(cabin_cat)
cabin_cat.describe()

The last output shows the "Cabin" variable condensed into broader categories, which is the letter that represents the location on the ship. Moreover, we can see that 687 passengers did not have a cabin listed, because it comes from the "NaN" values. This represents more than 65% of all the passengers. With so many missing values in this category, perhaps this variable does not contain much useful information. Nevertheless, a passenger with his or her cabin missing could mean that he or she died. Hence, let's keep this new categorized variable in the data frame.

In [ ]:
titanic["Cabin"]=cabin_cat

Checking the head of the data frame once more:

In [ ]:
titanic.head()

The next important step is to analyze whether there are missing values and outliers. The first column that we are going to analyze is the "Age" variable:

In [ ]:
titanic["Age"].describe()

If we remember from above, the number of passengers listed in this data frame was 891, but we only have 714 ages listed in the data frame. This is a clear sign that we have missing values. We can find out which rows contain missing values in the "Age" category with the numpy function np.where():

In [ ]:
missing_age = np.where(titanic["Age"].isnull()==True)
missing_age

We can also find out the quantity of missing values in this category:

In [ ]:
len(missing_age[0])

Since the number of passengers is almost 900 in the data frame and we have 177 missing values for the age, it would probably not be a good idea to delete passengers' age altogether. We could replace the missing values with zeros, or with a statistical value such as the mean or median.

The option of replacing the missing values with zeros is not a good one for our case, since a person's age cannot be 0. A simple fix like setting the missing values of age to be equal to some central value is interesting, but first we have to check the distribution of ages in the data frame. So, let's create a histogram of the passengers' ages:

In [ ]:
grid=sns.FacetGrid(titanic,height=5)
grid.map(plt.hist,'Age',bins=20)
grid.add_legend()

From the histogram, we can see that the most common ages are between 20 and 30. Hence,  we could fill in the missing ages with a central value between 20 and 30, such as 27:

In [ ]:
age_fill=np.where(titanic["Age"].isnull(),27,titanic["Age"])
titanic["Age"]=age_fill
titanic["Age"].describe()

We can check the same histogram again, since a considerable amount of values of 27 were added to the column:

In [ ]:
grid=sns.FacetGrid(titanic,height=5)
grid.map(plt.hist,'Age',bins=20)
grid.add_legend()

We can see now that the age distribution it is more centered between 20 and 30. It is definitely a better option than throwing entire records away of the passengers' ages.

We can also make the same missing value analysis on the "Embarked" column:

In [ ]:
titanic[titanic['Embarked'].isnull()==True]

From the table above, there are two entries in the DataFrame in which the passenger's port of entry in the ship was not listed. This is a considerably small number of missing values. Hence, we could fill these missing values with some center measure, such as the mode (most common value in the "Embarked" category in the dataset):

In [ ]:
titanic['Embarked'].mode()[0]

It looks like "S" is the most common port of entry. Therefore, we can replace the NaN values in the "Embarked" category with "S":

In [ ]:
titanic['Embarked']=titanic['Embarked'].fillna('S')

Rechecking the dataset to see if we still have null values in the "Embarked" column:

In [ ]:
titanic[titanic['Embarked'].isnull()==True]

The missing values in the "Embarked" category were dealt with accordingly.

We can now move on to outliers in the passengers' age distribution. If we analyze the previous histogram, we can see that a couple of passengers were very close to the age 80, which is very uncommon for this data set. However, we can observe that the majority of the values are centered around the mean, which is between 20 and 30. A box plot can be constructed for the "Age" category:

In [ ]:
plt.figure(figsize=(10,8))
titanic['Age'].plot(kind='box')

We can see from the plot above that there are a few outliers in the "Age" category. They are mostly represented by the infants (age very close to 0) and the people whose ages are above the midpoint between 50 and 60. The majority of the data lies in the region between 5 and 55.

Looking at the "Fare" variable, we can also investigate for outliers, by using a box plot:

In [ ]:
plt.figure(figsize=(10,8))
titanic['Fare'].plot(kind='box')

As we can clearly see from the box plot above, we have some outliers that are so far away from the whiskers that the box appears to be very small in the plot. This indicates that there are some passengers who paid fares that are much higher than the vast majority of the other passengers. We can investigate who are the people that paid such high fares:

In [ ]:
location=np.where(titanic["Fare"]==max(titanic["Fare"]))
titanic.loc[location]

Even though the box plot shows one passenger who paid more than all of the others, we can see, from the table above, that there were actually 3 passengers who paid a high fare. Since there are only 3 highrollers, we can leave the variable "Fare" unchanged.

Finally, we can discuss the need or not to create new variables. We can create a new variable called "Family", which combines the variables "Parch" and "SibSp", which can be used to indicate the total number of family members that a passenger brought to the Titanic, including siblings, parents, children and spouses:

In [ ]:
titanic["Family"]=titanic["SibSp"]+titanic["Parch"]

Checking the head of the data frame again:

In [ ]:
titanic.head()

We can also find out the people who had the most amount of family members on board:

In [ ]:
titanic_most_family=np.where(titanic["Family"]==max(titanic["Family"]))
titanic.loc[titanic_most_family]

As we can see from the table above, 7 people on board of the Titanic had 8 siblings or spouses and 2 parents or children.

Another category we can create regards the passengers who were alone on the ship (Family value of 0) and the ones who were accompanied by other people (Family value greater than 0). We can call this new category "IsAlone":

In [ ]:
titanic['IsAlone']=titanic['Family'].apply(lambda x: 1 if x==0 else 0)

Checking the head of the DataFrame with this new variable:

In [ ]:
titanic.head()

At this step, we can finally answer the questions. In order to answer question number 1, about what categories of passengers were most likely to survive the Titanic disaster, we can consider the categories "Sex", "Pclass", "Family", "Age", "Cabin", "Embarked" and "IsAlone" in terms of survival rate.

First, let's start with the class of passengers in order to see which class the majority of the passengers who survived belonged to:

In [ ]:
titanic.groupby('Pclass').sum()['Survived']

We can see that the majority of the people who survived belonged to Class 1. However, according to the original data frame, we know that there were many more people on board of the Titanic who belonged to Class 3 rather than Class 1. Hence, we can calculate the survival rate of the passengers in each class by dividing the amount of people who survived in that class by the total amount of passengers in that specific class:

In [ ]:
titanic.groupby('Pclass').sum()['Survived']/titanic.groupby('Pclass').count()['Survived']

We can observe that the highest survival rate is in Class 1, while the worst survival rate is in Class 3.

Now, let's move on to the "Sex" category:

In [ ]:
titanic.groupby('Sex').sum()['Survived']

In [ ]:
titanic.groupby('Sex').count()['Survived']

Calculating the survival rate of men and women on board of the Titanic:

In [ ]:
titanic.groupby('Sex').sum()['Survived']/titanic.groupby('Sex').count()['Survived']

We can observe that the highest survival rate is within females, while the worst survival rate is within males. This is expected, since the general policy in disasters is to save women and children first.

Now, let's move on to the "Family" category:

In [ ]:
titanic.groupby('Family').sum()['Survived']

In [ ]:
titanic.groupby('Family').count()['Survived']

Calculating the survival rate in terms of family size on board of the Titanic:

In [ ]:
titanic.groupby('Family').sum()['Survived']/titanic.groupby('Family').count()['Survived']

We can observe that the highest survival rate is within families whose size corresponds to a total of 3 people, while the worst survival rate is within families with 7 or 10 people on board.

Additionally, we can perform the same survival rate determination on the "IsAlone" variable, in order to obtain a more broad idea about the survival among people who were alone and the ones who were not:

In [ ]:
titanic.groupby('IsAlone').sum()['Survived']/titanic.groupby('IsAlone').count()['Survived']

We can see from the table above that a higher survival rate is observed among the people who were accompanied by at least one person, whereas a lower survival rate is observed among the people who were alone. This result is also expected, since it is likely that the people who were not alone and also survived the Titanic disaster probably went to a life boat accompanying his or her spouse, or children. The people who were alone, especially men, were less likely to survive, because probably they were not given priority on the life boats.

The survival rate can also be determined among the "Cabin" category. Ignoring the missing values (values marked with an "n") on the column, we can perform the following:

In [ ]:
by_cabin_survival=titanic.groupby('Cabin').sum()['Survived']/titanic.groupby('Cabin').count()['Survived']
by_cabin_survival.drop('n',inplace=True)
by_cabin_survival

The highest survival rates are observed on the cabin locations B, D and E on the ship.

In order to check the survival rate in terms of age on board of the Titanic, we can construct histograms:

In [ ]:
grid=sns.FacetGrid(titanic,col='Survived',height=5)
grid.map(plt.hist,'Age',bins=20)
grid.add_legend()

We can disregard the peak rectangle in both graphs, because we filled the NaN values for "Age" with the number 27. Additionally, we can observe that the majority of individuals who survived (right graph) were babies and infants, along with passengers between the ages of 30 and 40. The majority of individuals who did not survive (left graph) were people in their twenties and thirties.

We can explore the "Age" category additionally, by making age bands, starting from 0, going up to 80 (maximum age value from the previous histograms). Then, the age bands can be grouped for all passengers and a correlation with the "Survived" category can be established:

In [ ]:
bins=[0,16,32,48,64,80]
titanic['AgeBand']=pd.cut(titanic['Age'],bins,right=True)
ageband=titanic[['AgeBand','Survived']].groupby(['AgeBand'],as_index=False).mean().sort_values(by='AgeBand',ascending=True)
ageband

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(x='AgeBand',y='Survived',data=ageband)

As it can clearly be seen from the table and bar graph above, the best survival rate, in terms of age, was achieved by the age band of 0 to 16, which includes babies, infants and teenagers. The worst survival rate is observed on the last age band (elderly), between the ages of 64 and 80.

The survival rate can also be calculated for the "Embarked" column, in order to reveal which was the port of entry for which most of the passengers on board of the Titanic survived:

In [ ]:
titanic.groupby('Embarked').sum()['Survived']/titanic.groupby('Embarked').count()['Survived']

From the table above, we can observe that the port of entry in the ship which showed the highest survival rate was the "C" port, whereas the one that showed the worst survival rate was the "S" port.

Now, we can construct graphs that visually represent the tables shown above, in order to answer the first question:

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(x='Survived',data=titanic,hue='Pclass')

We can see the same pattern discussed before regarding classes in the ship. A considerable amount of people from Class 1 survived in comparison with the amount of passengers from the same class that perished. In Class 2, fewer people survived than in Class 1, and more died. In Class 3, this trend also happens, but with more intensity. Only a few more than one hundred survived, whereas more than 350 perished.

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(x='Survived',data=titanic,hue='Sex')

It can be seen in the plot above that many more women survived than men, even though the total number of women on board was lower than the total number of men.

In [ ]:
grid=sns.FacetGrid(titanic,row='Survived',col='Sex',height=5)
grid.map(plt.hist,'Age',bins=20)
grid.add_legend()

Observing the histograms above, we can say that, among females, the main age distribution of those who survived was between 15 and 30, whereas, for the men, this age distribution was between 0 and 10 and 20 and 30.

Additionally, we can use the crosstab() method to organize a table with the age bands created before in this analysis as the index and the gender of the passengers as the columns, in order to find the survival rate among the genders and the age bands:

In [ ]:
by_gender=pd.crosstab(index=titanic['AgeBand'],columns=titanic['Sex'],values=titanic['Survived'],aggfunc='mean')
by_gender

In [ ]:
by_gender[['female','male']].plot(kind='bar',figsize=(8,6))
plt.ylabel('Survival rate')

By observing the table and the bar graph above, we can conclude that the best survival rates, among women, was in the age band of 48 to 64 years of age, whereas, for the men, this value is in the age band of 0 to 16. The age band between 64 and 80 returned a null value for females because there was no woman listed in the dataset who belonged to this age category.

A similar method can be utilized to find the survival rates in terms of age band and class on the ship:

In [ ]:
by_class=pd.crosstab(index=titanic['AgeBand'],columns=titanic['Pclass'],values=titanic['Survived'],aggfunc='mean')
by_class

In [ ]:
by_class[['Class 1','Class 2','Class 3']].plot(kind='bar',figsize=(8,6))
plt.ylabel('Survival rate')

From the table and the graph above, it can be observed that, for first class, the best survival rate was among infants, children and teenagers, while the worst was among the elderly (64-80 age band). This trend also occurs for the second and third classes.

Family size can also be included in this crosstab() method in terms of survival rate:

In [ ]:
by_fam_size=pd.crosstab(index=titanic['AgeBand'],columns=titanic['Family'],values=titanic['Survived'],aggfunc='mean')
by_fam_size

In [ ]:
by_fam_size[[0,1,2,3,4,5,6,7,10]].plot(kind='bar',figsize=(10,8))
plt.ylabel('Survival rate')

The table and graph above show that the best survival rate is observed among people from the 16-32 age band, with 6 being their total family size (including children, spouse, siblings and parents), and also among people from the 48-64 age band, with a family size of 4. The worst survival rate is seen among the elderly (64 to 80 years of age), with a total family size of 1 and 2. The missing values represent absence of data in those regions of family size and age band.

For the "Embarked" feature:

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(x='Survived',data=titanic,hue='Embarked')

Among the ports of entry in the ship, the category "S" has the worst survival rate, since only a little over 200 people from this type of port of entry survived, compared to more than 400 who persihed. The port location "C" showed the greatest chances of survival, since more than 80 people survived and less than 80 persished, which represents a survival rate of more than 50%.

In [ ]:
# ANSWER TO QUESTION NUMBER 1:

The people who were most likely to survive the Titanic disaster were people from Class 1, females, babies, infants and passengers between the ages of 20 and 30 and in the range of 0-16, along with passengers who were not alone on the ship (accompanied by family members). In terms of gender, the highest survival rate was observed among females. When it comes to class, the highest survival rate was seen among passengers from Class 1. 
It is only logical that the highest survival rate was observed among people from Class 1, since the location of their cabins in the ship was higher than the location of the cabins in Class 2 and Class 3. Hence, it was easier for people from Class 1 to escape the flooding vessel and reach the lifeboats. Babies, infants and females showed a higher survival rate than men because they are usually saved first in a crisis situation. Therefore, females who belonged in Class 1 showed a much higher survival rate than men from Class 3. Despite the fact that there were a lot of missing values in the age category, the highest survival rate was observed in young passengers, which makes sense, since they usually show better mobility and sharper senses to deal with emergency and high-pressure situations. Among the "Embarked" feature, the highest survival rate was observed among the passengers who embarked the ship through the "C" port, whereas the worst survival rate was observed for the "S" port of entry.
A higher survival rate was observed among people who were accompanied by at least one person, while the people who were alone on the ship were less likely to survive. Finally, the highest survival rates, in terms of cabin location on the ship, were observed on the B, D and E levels, whereas the worst are on the A, G and T levels.

In [ ]:
# ANSWER TO QUESTION NUMBER 2:

The main attributes used in the analysis of the Titanic dataframe were: age, class, family size, gender (or sex), cabin, the "survived" attribute, fare and embarked features.
Despite missing some values, the "Age" category was important for the analysis, because it revealed the mean age of the passengers who survived the Titanic tragedy, even before the adjustment of filling the missing values with a center value for the age. It also revealed the age distribution of the passengers who survived and the ones who did not. Histograms were constructed to reveal the information mentioned above.
Class was also a very important attribute in the analysis, since it showed a trend that passengers who bought tickets for first class had a much better survival rate than the ones who were in third class, due to the location of the respective cabins in the ship. The dataframe was grouped by class, and ratios of passengers who survived the accident to the total number of passengers for each class were calculated in order to determine the survival rate of each class in the ship. Additionally, countplots with seaborn were constructed to visually represent the survival rates for the classes.
Family size was a new attribute used in the analysis. It involved the sum of the categories "Parch" and "SibSp", in order to reveal the quantity of family members that each passenger had on board of the Titanic. A new column was contructed in the Titanic dataframe, called "Family", in order to show the total number of family members per passenger. Its importance lies on the fact that the best survival rates were found to belong to small families, ideally three members (probably parents and a child). The worst survival rates were found to be among the big families (10 members), since these were probably in third class, whereas the small families were probably distributed in the first and second classes.
Gender or sex was equally important, because it revealed that the best survival rates lied among women, and the worst ones among men, probably due to human nature in face of disasters: women and children are usually saved first. Ratios between the number of women who survived the Titanic tragedy and the total number of women present in the ship (this was also done for the men) were calculated, as well as smaller dataframes using the "groupby" method with respect to gender were constructed. To visually represent what the dataframe showed, histograms were built comparing gender and survival rate.
The cabin attribute was first transformed, in order to reveal the general location in the ship, so the numbers after the first letters were removed. Nevertheless, there were a lot of missing values in the "Cabin" category, so its relevance was not as much as the relevance of the other attributes mentioned in this analysis.
Finally, the obvious "Survived" attribute was used, in order to determine the survival rate of the passengers in the other attributes aforementioned. Since this was a dummy variable (0 for absence or death and 1 for presence or survival), it was considerably easy to work with this attribute, especially to add the number of passengers who survived and the ones who died in the tragedy and then construct all the tables seen in the analysis. Additionally, this attribute made the construction of the histograms possible, always separating into at least two graphs: one where "Survived" was equal to 0 and another where "Survived" was equal to 1.
The "Fare" attribute showed high dispersion and some outliers, since there was the presence of a small group of people who paid considerably more than the majority of the passengers on board of the Titanic. A box plot was constructed in order to visualize the outliers and a table was constructed to find out who paid such high fares.
The "IsAlone" and "AgeBand" categories showed valuable information in terms of survival rate. The pd.cut() method was used to separate the passengers among age bins, while the "IsAlone" variable was constructed by analyzing whether the "Family" category was 0 or different than 0. The crosstab() method was used to make pivot tables of different categories (class, family size and gender) with the age bands when it comes to survival rate.

In [ ]:
# ANSWER TO QUESTION NUMBER 3:

Three attributes were created in the Titanic data frame. The "Family" category combined the variables "Parch" and "SibSp", which can be used to indicate the total number of family members that a passenger brought to the Titanic, including siblings, parents, children and spouses. This helped tremendously in the analysis of survival rate in terms of family size and reduced the size of the analysis, since analyzing parents and children and then siblings and spouses would be more time consuming and rather pointless, because we cannot know for sure if a certain figure in the "Parch" category represents a determinate amount of children or parents. 
The "IsAlone" category was created with the purpose of dividing the passengers in two groups: the ones who were alone on the Titanic and the ones who were not. This allowed a more broad analysis on the survival rate.
The "AgeBand" feature was created with the purpose of grouping the passengers in terms of age bins, and to establish a correlation with the "Survived" feature.
Some attributes were excluded from the analysis. They were the passenger ID and the ticket number. The reason why "PassengerID" was excluded from the data frame is because it is just a number assigned to each passenger, much like a serial variable in SQL, so it is nothing more than an arbitrary identifier. The reason why "Ticket" was also excluded is because it showed 681 unique values, while the count in the column was 891. The number of unique tickets was not close to the number of passengers, and the ticket numbers did not appear to follow any pattern we could use for grouping people in different categories, it just represented their ticket number that they bought in order to board the Titanic.

In [ ]:
# ANSWER TO QUESTION NUMBER 4:

There were two main categories with missing values: "Cabin" and "Age". The variable "Cabin" was modified in order to show only the letter and not the numbers, because the primary focus is the general location of the cabins of the passengers in the ship, and not the specific number of each cabin. There were a lot of missing values in the "Cabin" category, but this variable was not excluded from the data frame, because a passenger with his or her cabin missing could mean that he or she died; after all, how would we know what cabin a passenger stayed in if they were not around to tell the tale?
The "Age" category also contained a number of missing values, but it was not excluded from the Titanic data frame, since total deletion of this category would implicate in a significant loss of data. Instead, a histogram was contructed in order to discover what was the mean age of the passengers, which stayed between 20 and 30. A mean value of 27 was chosen and then the missing values in the "Age" category were filled with this central value, since a value of zero was not possible in this case.
In addition to the categories "Age" and "Cabin", the column "Embarked" also showed two missing values. Since the amount of missing values in this category was considerably low, they were replaced by the mode or most common value in the column, which was port of entry "S". The mode was chosen because it is a good central measurement to be used for nominal variables.